<a href="https://colab.research.google.com/github/Harman717/ViralSense/blob/development/ViralSense.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install python-arango arango-datasets


In [6]:
!pip show python-arango

Name: python-arango
Version: 8.1.6
Summary: Python Driver for ArangoDB
Home-page: https://github.com/arangodb/python-arango
Author: 
Author-email: Joohwan Oh <joohwan.oh@outlook.com>
License: MIT License

Copyright (c) 2016-2021 Joohwan Oh

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEM

In [7]:
!pip uninstall python-arango
!pip install python-arango

Found existing installation: python-arango 8.1.6
Uninstalling python-arango-8.1.6:
  Would remove:
    /usr/local/lib/python3.11/dist-packages/arango/*
    /usr/local/lib/python3.11/dist-packages/python_arango-8.1.6.dist-info/*
Proceed (Y/n)? y
  Successfully uninstalled python-arango-8.1.6
  Using cached python_arango-8.1.6-py3-none-any.whl.metadata (8.2 kB)
Using cached python_arango-8.1.6-py3-none-any.whl (114 kB)


In [8]:
!pip install nx_arangodb

In [9]:
from arango.client import ArangoClient

# Initialize the ArangoDB client with the correct host
client = ArangoClient(hosts='https://b8db61641c5b.arangodb.cloud:8529')
#NOTE : Need to confirm if using system is valid or should create a use viral sense
# Connect to the database
db = client.db('_system', username='root', password='HRJK7gcb7QqfIqPz7Ly9')

# Print database version (to check if connection is successful)
print(db.version())


3.12.4


In [10]:
from arango_datasets import Datasets

# Initialize the Datasets utility
datasets = Datasets(db)

# Load the Synthea P100 dataset
datasets.load('SYNTHEA_P100')
#NOTE: handle exceptions if dataset fails to load.

Output()

Output()

Initializing collection 'patients'


Output()

Output()

...

Initializing collection 'patients_to_medications'


Output()

Output()

Initializing collection 'encounters_to_imaging_studies'


Output()

In [11]:
import networkx as nx
# Get the graph from ArangoDB
graph = db.graph('SYNTHEA_P100')

# Get all vertex collections (nodes)
vertex_collections = graph.vertex_collections()

# Get all edge collections
edge_collections = graph.edge_definitions()

In [12]:
# Create a NetworkX Graph
G = nx.Graph()

# Load nodes into NetworkX
for collection in vertex_collections:
    cursor = db.collection(collection).all()
    for document in cursor:
        G.add_node(document['_key'], **document)  # Add node with attributes

# Load edges into NetworkX
for edge_collection in edge_collections:
    cursor = db.collection(edge_collection['edge_collection']).all()
    for edge in cursor:
        G.add_edge(edge['_from'].split('/')[1], edge['_to'].split('/')[1], **edge)  # Add edge with attributes

# Print summary of the NetworkX graph
print(f"Graph has {G.number_of_nodes()} nodes and {G.number_of_edges()} edges")

Graph has 114167 nodes and 311132 edges


In [13]:
# Calculate the degree of a specific node (replace 107 with the desired node key)
# Note: need to confirm what value should be here
node_degree = G.degree('104')
print(f"The degree of node 104 is: {node_degree}")

The degree of node 104 is: 16


In [14]:
import nx_arangodb as nxadb  # Import the nx-arangodb package

# Assuming you already have a NetworkX graph `G` for Synthea P100 loaded
# (from your previous steps)

# Create an ArangoDB graph from the NetworkX graph `G`
G_adb = nxadb.Graph(
    name="Synthea_P100",          # The name of the graph to be stored in ArangoDB
    db=db,                        # The ArangoDB database connection (from your earlier setup)
    incoming_graph_data=G,        # The NetworkX graph (G) you created from Synthea P100 dataset
    write_batch_size=50000,        # Optional: Set a batch size for bulk writes (modify as needed)
    overwrite_graph=True
)

# Print the created graph to confirm
print(G_adb)

/usr/local/lib/python3.11/dist-packages/cupy/_environment.py:541: UserWarning: 
--------------------------------------------------------------------------------

  CuPy may not function correctly because multiple CuPy packages are installed
  in your environment:

    cupy-cuda11x, cupy-cuda12x

  Follow these steps to resolve this issue:

    1. For all packages listed above, run the following command to remove all
       existing CuPy installations:

         $ pip uninstall <package_name>

      If you previously installed CuPy via conda, also run the following:

         $ conda uninstall cupy

    2. Install the appropriate CuPy package.
       Refer to the Installation Guide for detailed instructions.

         https://docs.cupy.dev/en/stable/install.html

--------------------------------------------------------------------------------

  warnings.warn(f'''
[20:32:20 +0000] [INFO]: NetworkX-cuGraph is unavailable: No module named 'cuda'.
INFO:nx_arangodb:NetworkX-cuGraph is unava

Output()

Output()

[2025/03/09 20:32:41 +0000] [4477] [INFO] - adbnx_adapter: Created ArangoDB 'Synthea_P100' Graph
INFO:adbnx_adapter:Created ArangoDB 'Synthea_P100' Graph


Graph named 'Synthea_P100' with 114167 nodes and 0 edges


In [15]:
G_adb = nxadb.Graph(name="Synthea_P100", db=db)

print(G_adb)

[20:32:54 +0000] [INFO]: Graph 'Synthea_P100' exists.
INFO:nx_arangodb:Graph 'Synthea_P100' exists.
[20:32:54 +0000] [INFO]: Default node type set to 'Synthea_P100_node'
INFO:nx_arangodb:Default node type set to 'Synthea_P100_node'


Graph named 'Synthea_P100' with 114167 nodes and 11131 edges


In [16]:
collections = db.collections()

Step 4

In [17]:
!pip install langchain


In [18]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [19]:
!apt-get update
!apt-get install -y --no-install-recommends \
    build-essential \
    cmake \
    libboost-all-dev \
    libnuma-dev

# Install RAPIDS
!pip install cudf-cu11 dask-cudf-cu11 cuml-cu11 cugraph-cu11 --extra-index-url=https://pypi.nvidia.com

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [20]:
!pip install langchain-community

In [21]:
!pip install -U langchain-openai
!pip install --upgrade langchain

In [22]:
from langchain_community.graphs.arangodb_graph import ArangoGraph

arango_graph = ArangoGraph(db)

In [23]:
!pip install cugraph-cu11 dask-cudf-cu11 cuml-cu11 cugraph-cu11 --extra-index-url=https://pypi.nvidia.com
!pip install langchain fastapi uvicorn openai

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.8 MB/s eta 0:00:00


In [34]:
!pip install fastapi uvicorn pyngrok nest_asyncio

In [45]:
import os
import re
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

# ----------------------- LangChain & OpenAI Setup -----------------------
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from google.colab import userdata

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
if not OPENAI_API_KEY:
    raise ValueError("Please set your OPENAI_API_KEY in your environment.")

llm = ChatOpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

# ----------------------- AQL Query Generation -----------------------
# Updated prompt: instructs to join medications with patient records.
aql_prompt_template = PromptTemplate(
    input_variables=["nl_query"],
    template="""
You are an expert in ArangoDB and AQL. The database has the following collections:
- patients: Contains patient demographic and healthcare information.
- conditions: Patient conditions and diagnoses.
- encounters: Patient encounters and procedures.
- patients_to_encounters: Relationships between patients and encounters.
- medications: Contains medication records.
- encounters_to_medications: Relationships between encounters and medications.
- devices, organizations, payers, procedures, supplies, allergies, immunizations, observations, and their relationships.
Generate an optimized AQL query for the following natural language query:
"{nl_query}"
Ensure that the query retrieves both the patient records and their corresponding medication details by appropriately joining the medications (or encounters_to_medications) collection.
Provide only the valid AQL query as output.
"""
)

def generate_aql(nl_query):
    prompt = aql_prompt_template.format(nl_query=nl_query)
    result = llm(prompt)
    if hasattr(result, "content"):
        result = result.content
    aql_query = result.strip()
    # Remove markdown formatting if present.
    aql_query = re.sub(r"^```(.*?)```$", r"\1", aql_query, flags=re.DOTALL).strip()
    return aql_query

# ----------------------- Personalized Summarization -----------------------
# Standard summarization prompt for simple/complex/hybrid queries.
summarization_prompt_template = PromptTemplate(
    input_variables=["query", "results"],
    template="""
You are an expert data analyst with a knack for storytelling. Interpret the following analytics results from a healthcare graph query and provide a concise, personalized summary that replaces raw patient IDs with friendly names.
If the results include medication information for each patient, include those details in your summary. For example, your answer might be:
"Frankie174 Schinner682 is taking metformin and insulin, while Stefanie819 Hessel84 is on glipizide and lisinopril. Overall, the medications indicate a focus on blood sugar control."
Query: "{query}"
Raw Results: {results}
Provide your summary below:
"""
)

def summarize_results_func(query, results):
    prompt = summarization_prompt_template.format(query=query, results=str(results))
    summary = llm(prompt)
    if hasattr(summary, "content"):
        summary = summary.content
    return summary.strip()

# ----------------------- Epidemic Analytics Summarization -----------------------
epidemic_summarization_prompt_template = PromptTemplate(
    input_variables=["query", "results"],
    template="""
You are an expert in epidemiology and graph analytics. Interpret the following epidemic analytics results derived from a healthcare graph.
The results include communities of patients (each community is a set of friendly names) and, for each community, a patient identified as a potential super-spreader (the one with the highest betweenness centrality).
Provide a clear, engaging, and actionable summary that highlights potential outbreak clusters and early-warning signals. For example:
"In Community A, including Alice, Bob, and Charlie, Bob appears to be a super-spreader and this cluster is at elevated risk for outbreak; early intervention is recommended."
Query: "{query}"
Raw Epidemic Analytics Results: {results}
Provide your summary below:
"""
)

def summarize_epidemic_results_func(query, epidemic_results):
    prompt = epidemic_summarization_prompt_template.format(query=query, results=str(epidemic_results))
    summary = llm(prompt)
    if hasattr(summary, "content"):
        summary = summary.content
    return summary.strip()

# ----------------------- GPU-Accelerated Analytics -----------------------
try:
    import cugraph
    import cudf
    use_cugraph = True
    print("cuGraph is available. Using GPU-accelerated analytics.")
except ImportError:
    use_cugraph = False
    print("cuGraph not available. Falling back to NetworkX analytics.")

def compute_pagerank(nx_graph):
    if use_cugraph:
        edges = [(u, v) for u, v in nx_graph.edges()]
        df = pd.DataFrame(edges, columns=["src", "dst"])
        g_df = cudf.DataFrame.from_pandas(df)
        cuG = cugraph.Graph()
        cuG.from_cudf_edgelist(g_df, source='src', destination='dst', renumber=True)
        pr_result = cugraph.pagerank(cuG)
        pr_result = pr_result.to_pandas().sort_values(by="pagerank", ascending=False)
        return pr_result.head(5).to_dict(orient='records')
    else:
        pr = nx.pagerank(nx_graph)
        pr_sorted = sorted(pr.items(), key=lambda x: x[1], reverse=True)
        return [{"node": node, "pagerank": rank} for node, rank in pr_sorted[:5]]

# ----------------------- Base Agentic Application -----------------------
class AgenticApp:
    def __init__(self, db, arango_graph, nx_graph):
        self.db = db
        self.arango_graph = arango_graph
        self.nx_graph = nx_graph

    def summarize_results(self, query, results):
        return summarize_results_func(query, results)

    def process_simple_query(self, query):
        aql_query = generate_aql(query)
        try:
            result_cursor = self.db.aql.execute(aql_query)
            result_list = list(result_cursor)
            if not result_list:
                return "No matching records were found."
            else:
                summary = self.summarize_results(query, result_list)
                return summary
        except Exception as e:
            return f"Error executing AQL query: {e}"

    def process_complex_query(self, query):
        analytics_results = compute_pagerank(self.nx_graph)
        # Map friendly names for each record using name_map.
        for entry in analytics_results:
            node_id = entry.get("node")
            entry["name"] = self.name_map.get(node_id, f"Patient {node_id[-4:]}")
        return self.summarize_results(query, analytics_results)

# ----------------------- Utility: Build Subgraph from AQL Results -----------------------
def build_subgraph_from_results(result_list):
    subG = nx.Graph()
    for record in result_list:
        patient_doc = record.get("patient")
        if patient_doc and patient_doc.get("_key"):
            patient_id = patient_doc["_key"]
            # Save medications as an attribute if available.
            medications = record.get("medications", [])
            patient_doc["medications"] = medications
            subG.add_node(patient_id, **patient_doc)
            for condition in record.get("conditions", []):
                if condition.get("_key"):
                    cond_id = condition["_key"]
                    subG.add_node(cond_id, **condition)
                    subG.add_edge(patient_id, cond_id)
            for med in medications:
                if isinstance(med, dict) and med.get("_key"):
                    med_id = med["_key"]
                    subG.add_node(med_id, **med)
                    subG.add_edge(patient_id, med_id)
                elif isinstance(med, str):
                    subG.add_edge(patient_id, med)
    return subG

# ----------------------- Enhanced Epidemic Analytics -----------------------
# New method to process epidemic-related queries.
class EnhancedAgenticApp(AgenticApp):
    def __init__(self, db, arango_graph, nx_graph, name_map):
        super().__init__(db, arango_graph, nx_graph)
        self.name_map = name_map  # Mapping of node IDs to friendly names.
        self.query_router = QueryRouter()

    def process_epidemic_query(self, query):
        aql_query = generate_aql(query)
        try:
            result_cursor = self.db.aql.execute(aql_query)
            result_list = list(result_cursor)
        except Exception as e:
            return f"Error executing AQL query: {e}"
        if not result_list:
            return f"No matching records were found. Generated AQL query was: {aql_query}"

        subgraph = build_subgraph_from_results(result_list)
        # Compute communities using greedy modularity.
        from networkx.algorithms import community
        communities = list(community.greedy_modularity_communities(subgraph))
        # Compute betweenness centrality for potential super-spreader detection.
        betweenness = nx.betweenness_centrality(subgraph)
        community_results = []
        for idx, comm in enumerate(communities):
            # Identify node with highest betweenness within this community.
            comm_bet = {node: betweenness[node] for node in comm if node in betweenness}
            if comm_bet:
                super_spreader = max(comm_bet, key=comm_bet.get)
            else:
                super_spreader = None
            # Convert node IDs to friendly names.
            friendly_comm = [self.name_map.get(node, f"Patient {node[-4:]}") for node in comm]
            friendly_super = self.name_map.get(super_spreader, f"Patient {super_spreader[-4:]}" ) if super_spreader else "N/A"
            community_results.append({
                "community_id": idx + 1,
                "members": friendly_comm,
                "super_spreader": friendly_super,
                "super_spreader_score": comm_bet.get(super_spreader, 0) if super_spreader else 0
            })
        epidemic_results = {
            "communities": community_results,
            "total_communities": len(communities)
        }
        summary = summarize_epidemic_results_func(query, epidemic_results)
        return summary

    def process_query(self, query):
        query_type = self.query_router.route(query)
        if query_type == "epidemic":
            return self.process_epidemic_query(query)
        elif query_type == "simple":
            return self.process_simple_query(query)
        elif query_type == "complex":
            return self.process_complex_query(query)
        else:
            return self.process_hybrid_query(query)

    def process_hybrid_query(self, query):
        # For queries that don't fall into the above, use hybrid logic.
        aql_query = generate_aql(query)
        try:
            result_cursor = self.db.aql.execute(aql_query)
            result_list = list(result_cursor)
        except Exception as e:
            return f"Error executing AQL query: {e}"
        if not result_list:
            return f"No matching records were found. Generated AQL query was: {aql_query}"

        subgraph = build_subgraph_from_results(result_list)
        analytics_results = compute_pagerank(subgraph)
        for entry in analytics_results:
            node_id = entry.get("node")
            entry["name"] = self.name_map.get(node_id, f"Patient {node_id[-4:]}")
        return self.summarize_results(query, analytics_results)

# ----------------------- Query Routing -----------------------
class QueryRouter:
    def route(self, nl_query):
        # Check for epidemic-related keywords.
        if any(keyword in nl_query.lower() for keyword in ["outbreak", "super-spreader", "risk", "predict", "epidemic", "early warning"]):
            return "epidemic"
        decision_prompt = (
            f"Decide if the query: '{nl_query}' requires a hybrid AQL and graph analytics approach, a simple AQL query, or a complex analytics query. "
            "Answer with 'hybrid', 'simple', or 'complex'."
        )
        decision = llm(decision_prompt)
        if hasattr(decision, "content"):
            decision = decision.content
        decision = decision.strip().lower()
        if "hybrid" in decision:
            return "hybrid"
        elif "complex" in decision:
            return "complex"
        else:
            return "simple"

# ----------------------- Build Friendly Name Mapping -----------------------
name_map = {}
cursor = db.collection("patients").all()
for doc in cursor:
    if doc.get("FIRST") and doc.get("LAST"):
        full_name = f"{doc['FIRST']} {doc['LAST']}"
    else:
        full_name = doc["_key"]
    name_map[doc["_key"]] = full_name

# ----------------------- Instantiate the Enhanced Agentic App -----------------------
agentic_app = EnhancedAgenticApp(db, arango_graph, G, name_map)

# ----------------------- FastAPI Integration with ngrok -----------------------
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from pyngrok import ngrok
import nest_asyncio
import uvicorn
import threading

app = FastAPI()

class QueryRequest(BaseModel):
    query: str

@app.get("/")
async def root():
    return {"message": "ViralSense Agentic App is running!"}

@app.post("/query")
async def query_endpoint(request: QueryRequest):
    try:
        result = agentic_app.process_query(request.query)
        return {"result": result}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

# Set your ngrok authtoken (replace with your actual token)
ngrok.set_auth_token("2u5z8kjmm3CnfKGXGEpsYPVUWIJ_wM6RcUxAhrHnwExz7gh8")
public_url = ngrok.connect(8000)
print(f"🌐 Public URL: {public_url}")

nest_asyncio.apply()

def run_fastapi():
    uvicorn.run(app, host="0.0.0.0", port=8000)

thread = threading.Thread(target=run_fastapi, daemon=True)
thread.start()


cuGraph not available. Falling back to NetworkX analytics.
🌐 Public URL: NgrokTunnel: "https://1143-34-74-124-232.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [4477]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
